In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
credits = pd.read_csv("data/credits.csv",low_memory = False)
keywords = pd.read_csv("data/keywords.csv",low_memory = False)
movies_metadata = pd.read_csv("data/movies_metadata.csv",low_memory = False)

In [3]:
movies_metadata = movies_metadata[['id','genres','original_title','overview','production_companies']]
credits = credits[['id','cast','crew']]
keywords = keywords[['id','keywords']]
print(movies_metadata.shape)  

(45466, 5)


In [4]:
movies_metadata.isnull().sum()
credits.isnull().sum()
keywords.isnull().sum()

id          0
keywords    0
dtype: int64

In [5]:
movies_metadata['id'] = movies_metadata['id'].fillna(0)
movies_metadata['id']=pd.to_numeric(movies_metadata['id'], errors='coerce').astype('Int64')

merged_data = keywords.merge(movies_metadata, on = 'id')
merged_data = merged_data.merge(credits, on = 'id')

In [6]:
import ast
def toList(text):
    lst = []
    for i in ast.literal_eval(text):
            lst.append(i['name'])
    return lst

In [7]:
merged_data['genres'] = merged_data['genres'].apply(toList)
merged_data['keywords'] = merged_data['keywords'].apply(toList)

In [8]:
def getDir(obj):
    lst = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            lst.append(i['name'])
            break;
    return lst 

In [9]:
merged_data['crew'] = merged_data['crew'].apply(getDir)

In [10]:
merged_data.head(3)

,id,keywords,genres,original_title,overview,production_companies,cast,crew
0,862,"[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]",Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'cast_id': 14, 'character': 'Woody (voice)',...",[John Lasseter]
1,8844,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]",Jumanji,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'cast_id': 1, 'character': 'Alan Parrish', '...",[Joe Johnston]
2,15602,"[fishing, best friend, duringcreditsstinger, o...","[Romance, Comedy]",Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...",[Howard Deutch]


In [11]:
def castNames(text):
    list = []
    counter = 0
    for i in ast.literal_eval(text):
        if(counter<5):
            list.append(i['name'])
            counter+=1
        else:
            break
    return list

merged_data['cast'] = merged_data['cast'].apply(castNames)

In [12]:
merged_data.head(2)

,id,keywords,genres,original_title,overview,production_companies,cast,crew
0,862,"[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]",Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter]
1,8844,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]",Jumanji,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston]


In [13]:
merged_data['overview'] = merged_data['overview'].astype(str)
merged_data['overview'] = merged_data['overview'].apply(lambda x : x.split())

In [14]:
def collapse(list):
    collapsed_list = []
    for i in list:
        collapsed_list.append(i.replace(" ",""))
    return collapsed_list

In [15]:
merged_data['keywords'] = merged_data['keywords'].apply(collapse)
merged_data['genres'] = merged_data['genres'].apply(collapse)
merged_data['crew'] = merged_data['crew'].apply(collapse)
merged_data['cast'] = merged_data['cast'].apply(collapse)

In [16]:
merged_data

,id,keywords,genres,original_title,overview,production_companies,cast,crew
0,862,"[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]",Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[TomHanks, TimAllen, DonRickles, JimVarney, Wa...",[JohnLasseter]
1,8844,"[boardgame, disappearance, basedonchildren'sbo...","[Adventure, Fantasy, Family]",Jumanji,"[When, siblings, Judy, and, Peter, discover, a...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[RobinWilliams, JonathanHyde, KirstenDunst, Br...",[JoeJohnston]
2,15602,"[fishing, bestfriend, duringcreditsstinger, ol...","[Romance, Comedy]",Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ...","[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[WalterMatthau, JackLemmon, Ann-Margret, Sophi...",[HowardDeutch]
3,31357,"[basedonnovel, interracialrelationship, single...","[Comedy, Drama, Romance]",Waiting to Exhale,"[Cheated, on,, mistreated, and, stepped, on,, ...",[{'name': 'Twentieth Century Fox Film Corporat...,"[WhitneyHouston, AngelaBassett, LorettaDevine,...",[ForestWhitaker]
4,11862,"[baby, midlifecrisis, confidence, aging, daugh...",[Comedy],Father of the Bride Part II,"[Just, when, George, Banks, has, recovered, fr...","[{'name': 'Sandollar Productions', 'id': 5842}...","[SteveMartin, DianeKeaton, MartinShort, Kimber...",[CharlesShyer]
...,...,...,...,...,...,...,...,...
46623,439050,[tragiclove],"[Drama, Family]",رگ خواب,"[Rising, and, falling, between, a, man, and, w...",[],"[LeilaHatami, KouroshTahami, ElhamKorda]",[HamidNematollah]
46624,111109,"[artist, play, pinoy]",[Drama],Siglo ng Pagluluwal,"[An, artist, struggles, to, finish, his, work,...","[{'name': 'Sine Olivia', 'id': 19653}]","[AngelAquino, PerryDizon, HazelOrencio, JoelTo...",[LavDiaz]
46625,67758,[],"[Action, Drama, Thriller]",Betrayal,"[When, one, of, her, hits, goes, wrong,, a, pr...","[{'name': 'American World Pictures', 'id': 6165}]","[ErikaEleniak, AdamBaldwin, JulieduPage, James...",[MarkL.Lester]
46626,227506,[],[],Satana likuyushchiy,"[In, a, small, town, live, two, brothers,, one...","[{'name': 'Yermoliev', 'id': 88753}]","[IwanMosschuchin, NathalieLissenko, PavelPavlo...",[YakovProtazanov]


In [17]:
merged_data['collection'] = merged_data['keywords'] + merged_data['genres'] + merged_data['overview'] + merged_data['cast'] + merged_data['crew']

In [18]:
new_data = merged_data[['id','original_title', 'collection']].head(25000).copy()
new_data

,id,original_title,collection
0,862,Toy Story,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,Jumanji,"[boardgame, disappearance, basedonchildren'sbo..."
2,15602,Grumpier Old Men,"[fishing, bestfriend, duringcreditsstinger, ol..."
3,31357,Waiting to Exhale,"[basedonnovel, interracialrelationship, single..."
4,11862,Father of the Bride Part II,"[baby, midlifecrisis, confidence, aging, daugh..."
...,...,...,...
24995,43193,Dangerous When Wet,"[musical, englishchannel, swimmer, swimming, c..."
24996,70512,L'inferno,"[hell, basedonsong,poemorrhyme, Adventure, Dra..."
24997,73103,Dante's Inferno,"[Animation, Comedy, A, darkly, comedic, travel..."
24998,38471,Dark Alibi,"[prison, bankrobbery, fingerprints, ex-con, Cr..."


In [19]:
new_data['collection'] = new_data['collection'].apply(lambda x: " ".join(x))

In [20]:
new_data['collection'] = new_data['collection'].apply(lambda x: x.lower())

In [21]:
new_data.head()

,id,original_title,collection
0,862,Toy Story,jealousy toy boy friendship friends rivalry bo...
1,8844,Jumanji,boardgame disappearance basedonchildren'sbook ...
2,15602,Grumpier Old Men,fishing bestfriend duringcreditsstinger oldmen...
3,31357,Waiting to Exhale,basedonnovel interracialrelationship singlemot...
4,11862,Father of the Bride Part II,baby midlifecrisis confidence aging daughter m...


In [22]:
import nltk

In [23]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [24]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [25]:
new_data['collection'] = new_data['collection'].apply(stem)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [27]:
vector = cv.fit_transform(new_data['collection']).toarray()
vector.shape

(25000, 5000)

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
similarity = cosine_similarity(vector)

In [30]:
def recommend(movie):
    index = new_data[new_data['original_title'] == movie].index[0]
    distances = similarity[index]
    most_similar_movies = sorted(list(enumerate(distances)),reverse=True,key = lambda x: x[1])[1:6]
    
    for i in most_similar_movies:
        print(new_data.iloc[i[0]].original_title)

In [32]:
recommend('Toy Story 2')

Toy Story
Toy Story 3
The Million Dollar Duck
Dollman vs. Demonic Toys
La freccia azzurra


In [36]:
import pickle
pickle.dump(new_data, open('final-movie.pkl', 'wb'))
pickle.dump(similarity,open('final-similarity.pkl', 'wb'),protocol = 4)

In [37]:
movies_metadata.head()

,id,genres,original_title,overview,production_companies
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",[{'name': 'Twentieth Century Fox Film Corporat...
4,11862,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'name': 'Sandollar Productions', 'id': 5842}..."


In [53]:
import requests
def fetch_poster(movie_id):
    url = "https://api.themoviedb.org/3/movie/550?api_key=4fcf7af3bbcf1cdb4598066485a155dd"
    response = requests.get(url)
    data = response.json()
    poster_path = data['poster_path']
    full_path = "https://image.tmdb.org/t/p/w500/" + poster_path
    return full_path 

In [54]:
print(fetch_poster(862))

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))